In [200]:
from datetime import datetime
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [201]:
import tensorflow_datasets as tfds

In [202]:
dataset, info = tfds.load("fashion_mnist", with_info=True, as_supervised=False)
info

tfds.core.DatasetInfo(
    name='fashion_mnist',
    version=1.0.0,
    description='Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.',
    homepage='https://github.com/zalandoresearch/fashion-mnist',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{DBLP:journals/corr/abs-1708-07747,
      author    = {Han Xiao and
                   Kashif Rasul and
                   Roland Vollgraf},
      title     = {Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning
                   Algorithms},
      journal   = {CoRR},
      volume

In [203]:
dataset, info = tfds.load(
    'fashion_mnist',
    as_supervised=True,
    split = [
        tfds.Split.TRAIN.subsplit(tfds.percent[:80]),
        tfds.Split.TRAIN.subsplit(tfds.percent[80:90]),
        tfds.Split.TRAIN.subsplit(tfds.percent[90:]),
    ],
    with_info=True
)

In [204]:
train_dataset = dataset[0]
test_dataset = dataset[1]
valid_dataset = dataset[2]

In [205]:
# Run this when as_supervised=False
# fig = tfds.show_examples(info, train_dataset)

In [206]:
for image, label in train_dataset.batch(32).take(2):
    print(type(image), type(label))
#     print(image.shape)

<class 'tensorflow.python.framework.ops.EagerTensor'> <class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'> <class 'tensorflow.python.framework.ops.EagerTensor'>


In [207]:
#Since the images and text are in string, we have to typecast them into float
def preprocess(image, label):
    image = tf.dtypes.cast(image, tf.float32)
    label = tf.dtypes.cast(label, tf.float32)
    
    image = tf.image.per_image_standardization(image)
    
    return image, label

In [208]:
train_dataset = train_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)
valid_dataset = valid_dataset.map(preprocess)

In [209]:
for image, label in train_dataset.batch(2).take(2):
    print(image.shape, label)

(2, 28, 28, 1) tf.Tensor([3. 9.], shape=(2,), dtype=float32)
(2, 28, 28, 1) tf.Tensor([7. 6.], shape=(2,), dtype=float32)


In [210]:
# convulation layer 1
# 28x28x28
# 10 kernel
# stride 1
# filter_size = 3x3
# padding valid
# n_out x n_out x n_filter = 26x26x10
# filter_shape = 4D Tensor = [fHeight, fWidth, nChannels, numFilters] = [3,3,1,10]

In [211]:
# maxPool
# stride(2,2)
# poolSize = (2x2)
# out = 13x13x10

In [212]:
# convoltion layer 2
# input = 13x13x10
# 20 Kernels
# stride 1
# filter_size = (3x3)
# valid padding
# output = 11x11x20
# filterShape = 4D Tensor = [3,3,1,20]

In [213]:
# maxPool
# stride (2,2)
# poolSize = (2x2)
# out = 6x6x20

In [214]:
# Flattening
# input = 6x6x20
# output = 720x1

In [215]:
image.shape

TensorShape([2, 28, 28, 1])

In [216]:
class LeNet(tf.keras.Model):
    def __init__(self):
        super (LeNet, self).__init__()
        initializer = tf.initializers.GlorotUniform(seed=123)
        #Conv1
        self.wc1 = tf.Variable(initializer([3,3,1,10]), trainable=True, name='wc1')
        #Conv2
        self.wc2 = tf.Variable(initializer([3,3,10,20]), trainable=True, name='wc2')
        
        #Flattening
        
        #Dense
        #Adjusting the weights for the Dense Layer (ANN)
        self.wd3 = tf.Variable(initializer([500,128]), trainable=True)
        self.wd4 = tf.Variable(initializer([128,64]), trainable=True)
        self.wd5 = tf.Variable(initializer([64,10]), trainable=True)
        
        #Adjusting the Biases
        #Biases for the Covolution Layers
        self.bc1 = tf.Variable(tf.zeros([10]), dtype=tf.float32, trainable=True)
        self.bc2 = tf.Variable(tf.zeros([20]), dtype=tf.float32, trainable=True)
        
        #Biases for the Dense Layers
        self.bd3 = tf.Variable(tf.zeros([128]), dtype=tf.float32, trainable=True)
        self.bd4 = tf.Variable(tf.zeros([64]), dtype=tf.float32, trainable=True)
        self.bd5 = tf.Variable(tf.zeros([10]), dtype=tf.float32, trainable=True)
        
    def call(self,x):
        
        # 1st Convolutional Layer
        #layer equals NHWC
        #First the Convolutional layer and the MaxPool
        x = tf.nn.conv2d(x, self.wc1, strides=[1,1,1,1], padding="VALID")
        x = tf.nn.bias_add(x,self.bc1)
        x = tf.nn.relu(x)
        x = tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="VALID")
        
                
        # 2nd Convolutional Layer
        x = tf.nn.conv2d(x, self.wc2, strides=[1, 1, 1, 1], padding="VALID")
        x = tf.nn.bias_add(x, self.bc2)
        x = tf.nn.relu(x)
        x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
        
        # Flattten out
        # N X Number of Nodes
        # Flatten()
        x = tf.reshape(x, (tf.shape(x)[0], -1))
        
        # Dense1
        x = tf.matmul(x, self.wd3)
        x = tf.nn.bias_add(x, self.bd3)
        x = tf.nn.relu(x)

        
        # Dense2
        x = tf.matmul(x, self.wd4)
        x = tf.nn.bias_add(x, self.bd4)
        x = tf.nn.relu(x)
        
        
        # Dense3
        x = tf.matmul(x, self.wd5)
        x = tf.nn.bias_add(x, self.bd5)
#         x = tf.nn.sigmoid(x)
        
        return x

In [251]:
model = LeNet()

In [252]:
ce_loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [253]:
LEARNING_RATE = 0.001

In [254]:
optimizer = tf.optimizers.Adam(learning_rate=LEARNING_RATE)

In [255]:
model.compile(optimizer=optimizer,loss=ce_loss)

In [265]:
# model.fit(x=train_dataset, y=test_dataset, epochs=20)

In [256]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [221]:
def custom_optimizer(var, lr):
    return var - (var*lr)

In [222]:
# Calulating the Loss and applying the gradient descent for the respective variables that are trainable
def train_step(model, inputs, labels, loss_fn, optimizer):
    with tf.GradientTape() as t:

        y_predicted = model(inputs, training=True)
        current_loss = loss_fn(labels, y_predicted)
        
        gradients = t.gradient(current_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        return current_loss

In [223]:
def valid_step(model, inputs, labels, loss_fn):
    y_predicted = model(inputs, training=False)
    current_loss = loss_fn(labels, y_predicted)
    return current_loss

In [224]:
EPOCHS = 50
BATCH_SIZE = 32

In [225]:
train_dataset = train_dataset.batch(BATCH_SIZE)
valid_dataset = valid_dataset.batch(BATCH_SIZE)

In [226]:
sample = np.random.randn(2,28,28,1)

In [227]:
filters = tf.Variable(np.random.randn(3,3,1,10))

In [228]:
# tf.nn.conv2d(sample,filters,[1,1,1,1],padding="VALID")

In [229]:
len(model.trainable_variables)

10

In [230]:
losses = tf.keras.metrics.Mean(name='loss')
val_losses = tf.keras.metrics.Mean(name='val_loss')

##### Creating a CheckPoint

In [231]:
# Checkpoint
# callbacks = [tf.keras.Checkpoint(...)]
checkpoint_dir = f'./temp/train/checkpoints/'

In [232]:
optimizer = optimizer
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, net=model)
manager = tf.train.CheckpointManager(ckpt,checkpoint_dir,max_to_keep=None) #max_to_keep=None means keeping all the checkpoints

In [233]:
def train_and_checkpoint(model, manager):
    ckpt.restore(manager.latest_checkpoint)
    if manager.latest_checkpoint:
        print("Restored from {}".format(manager.latest_checkpoint))
    else:
        print("Initializing from scratch.")
        
    for x_batch, y_batch in train_dataset:
        loss = train_step(model, x_batch, y_batch, ce_loss, optimizer)
        ckpt.step.assign_add(1)
        
        if int(ckpt.step) % 10 == 0:
            save_path = manager.save()
            print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
            print("loss {:1.2f}".format(loss.numpy()))
        
        
# #   for example in toy_dataset():
# #     loss = train_step(model, example, optimizer)
# #     ckpt.step.assign_add(1)
# #     if int(ckpt.step) % 10 == 0:
# #       save_path = manager.save()
# #       print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
# #       print("loss {:1.2f}".format(loss.numpy()))

In [234]:
for epoch in range(EPOCHS):
    train_and_checkpoint(model, manager)

Initializing from scratch.
Saved checkpoint for step 10: ./temp/train/checkpoints/ckpt-1
loss 1.69
Saved checkpoint for step 20: ./temp/train/checkpoints/ckpt-2
loss 1.20
Saved checkpoint for step 30: ./temp/train/checkpoints/ckpt-3
loss 0.96
Saved checkpoint for step 40: ./temp/train/checkpoints/ckpt-4
loss 0.97
Saved checkpoint for step 50: ./temp/train/checkpoints/ckpt-5
loss 0.78
Saved checkpoint for step 60: ./temp/train/checkpoints/ckpt-6
loss 0.66
Saved checkpoint for step 70: ./temp/train/checkpoints/ckpt-7
loss 0.49
Saved checkpoint for step 80: ./temp/train/checkpoints/ckpt-8
loss 0.45
Saved checkpoint for step 90: ./temp/train/checkpoints/ckpt-9
loss 0.64
Saved checkpoint for step 100: ./temp/train/checkpoints/ckpt-10
loss 1.20
Saved checkpoint for step 110: ./temp/train/checkpoints/ckpt-11
loss 0.88
Saved checkpoint for step 120: ./temp/train/checkpoints/ckpt-12
loss 0.62
Saved checkpoint for step 130: ./temp/train/checkpoints/ckpt-13
loss 0.71
Saved checkpoint for step 140

Saved checkpoint for step 1120: ./temp/train/checkpoints/ckpt-112
loss 0.16
Saved checkpoint for step 1130: ./temp/train/checkpoints/ckpt-113
loss 0.47
Saved checkpoint for step 1140: ./temp/train/checkpoints/ckpt-114
loss 0.33
Saved checkpoint for step 1150: ./temp/train/checkpoints/ckpt-115
loss 0.53
Saved checkpoint for step 1160: ./temp/train/checkpoints/ckpt-116
loss 0.41
Saved checkpoint for step 1170: ./temp/train/checkpoints/ckpt-117
loss 0.43
Saved checkpoint for step 1180: ./temp/train/checkpoints/ckpt-118
loss 0.21
Saved checkpoint for step 1190: ./temp/train/checkpoints/ckpt-119
loss 0.22
Saved checkpoint for step 1200: ./temp/train/checkpoints/ckpt-120
loss 0.42
Saved checkpoint for step 1210: ./temp/train/checkpoints/ckpt-121
loss 0.56
Saved checkpoint for step 1220: ./temp/train/checkpoints/ckpt-122
loss 0.32
Saved checkpoint for step 1230: ./temp/train/checkpoints/ckpt-123
loss 0.27
Saved checkpoint for step 1240: ./temp/train/checkpoints/ckpt-124
loss 0.33
Saved checkp

Saved checkpoint for step 2200: ./temp/train/checkpoints/ckpt-220
loss 0.35
Saved checkpoint for step 2210: ./temp/train/checkpoints/ckpt-221
loss 0.27
Saved checkpoint for step 2220: ./temp/train/checkpoints/ckpt-222
loss 0.46
Saved checkpoint for step 2230: ./temp/train/checkpoints/ckpt-223
loss 0.11
Saved checkpoint for step 2240: ./temp/train/checkpoints/ckpt-224
loss 0.17
Saved checkpoint for step 2250: ./temp/train/checkpoints/ckpt-225
loss 0.18
Saved checkpoint for step 2260: ./temp/train/checkpoints/ckpt-226
loss 0.21
Saved checkpoint for step 2270: ./temp/train/checkpoints/ckpt-227
loss 0.33
Saved checkpoint for step 2280: ./temp/train/checkpoints/ckpt-228
loss 0.26
Saved checkpoint for step 2290: ./temp/train/checkpoints/ckpt-229
loss 0.17
Saved checkpoint for step 2300: ./temp/train/checkpoints/ckpt-230
loss 0.25
Saved checkpoint for step 2310: ./temp/train/checkpoints/ckpt-231
loss 0.27
Saved checkpoint for step 2320: ./temp/train/checkpoints/ckpt-232
loss 0.19
Saved checkp

Saved checkpoint for step 3280: ./temp/train/checkpoints/ckpt-328
loss 0.20
Saved checkpoint for step 3290: ./temp/train/checkpoints/ckpt-329
loss 0.25
Saved checkpoint for step 3300: ./temp/train/checkpoints/ckpt-330
loss 0.30
Saved checkpoint for step 3310: ./temp/train/checkpoints/ckpt-331
loss 0.21
Saved checkpoint for step 3320: ./temp/train/checkpoints/ckpt-332
loss 0.27
Saved checkpoint for step 3330: ./temp/train/checkpoints/ckpt-333
loss 0.31
Saved checkpoint for step 3340: ./temp/train/checkpoints/ckpt-334
loss 0.39
Saved checkpoint for step 3350: ./temp/train/checkpoints/ckpt-335
loss 0.13
Saved checkpoint for step 3360: ./temp/train/checkpoints/ckpt-336
loss 0.18
Saved checkpoint for step 3370: ./temp/train/checkpoints/ckpt-337
loss 0.27
Saved checkpoint for step 3380: ./temp/train/checkpoints/ckpt-338
loss 0.20
Saved checkpoint for step 3390: ./temp/train/checkpoints/ckpt-339
loss 0.33
Saved checkpoint for step 3400: ./temp/train/checkpoints/ckpt-340
loss 0.27
Saved checkp

Saved checkpoint for step 4360: ./temp/train/checkpoints/ckpt-436
loss 0.34
Saved checkpoint for step 4370: ./temp/train/checkpoints/ckpt-437
loss 0.12
Saved checkpoint for step 4380: ./temp/train/checkpoints/ckpt-438
loss 0.18
Saved checkpoint for step 4390: ./temp/train/checkpoints/ckpt-439
loss 0.15
Saved checkpoint for step 4400: ./temp/train/checkpoints/ckpt-440
loss 0.35
Saved checkpoint for step 4410: ./temp/train/checkpoints/ckpt-441
loss 0.21
Saved checkpoint for step 4420: ./temp/train/checkpoints/ckpt-442
loss 0.30
Saved checkpoint for step 4430: ./temp/train/checkpoints/ckpt-443
loss 0.08
Saved checkpoint for step 4440: ./temp/train/checkpoints/ckpt-444
loss 0.23
Saved checkpoint for step 4450: ./temp/train/checkpoints/ckpt-445
loss 0.20
Saved checkpoint for step 4460: ./temp/train/checkpoints/ckpt-446
loss 0.58
Saved checkpoint for step 4470: ./temp/train/checkpoints/ckpt-447
loss 0.16
Saved checkpoint for step 4480: ./temp/train/checkpoints/ckpt-448
loss 0.41
Saved checkp

...

Saved checkpoint for step 69790: ./temp/train/checkpoints/ckpt-6979
loss 0.01
Saved checkpoint for step 69800: ./temp/train/checkpoints/ckpt-6980
loss 0.00
Saved checkpoint for step 69810: ./temp/train/checkpoints/ckpt-6981
loss 0.00
Saved checkpoint for step 69820: ./temp/train/checkpoints/ckpt-6982
loss 0.02
Saved checkpoint for step 69830: ./temp/train/checkpoints/ckpt-6983
loss 0.10
Saved checkpoint for step 69840: ./temp/train/checkpoints/ckpt-6984
loss 0.00
Saved checkpoint for step 69850: ./temp/train/checkpoints/ckpt-6985
loss 0.12
Saved checkpoint for step 69860: ./temp/train/checkpoints/ckpt-6986
loss 0.01
Saved checkpoint for step 69870: ./temp/train/checkpoints/ckpt-6987
loss 0.02
Saved checkpoint for step 69880: ./temp/train/checkpoints/ckpt-6988
loss 0.00
Saved checkpoint for step 69890: ./temp/train/checkpoints/ckpt-6989
loss 0.03
Saved checkpoint for step 69900: ./temp/train/checkpoints/ckpt-6990
loss 0.00
Saved checkpoint for step 69910: ./temp/train/checkpoints/ckpt-6

Saved checkpoint for step 70840: ./temp/train/checkpoints/ckpt-7084
loss 0.02
Saved checkpoint for step 70850: ./temp/train/checkpoints/ckpt-7085
loss 0.00
Saved checkpoint for step 70860: ./temp/train/checkpoints/ckpt-7086
loss 0.00
Saved checkpoint for step 70870: ./temp/train/checkpoints/ckpt-7087
loss 0.19
Saved checkpoint for step 70880: ./temp/train/checkpoints/ckpt-7088
loss 0.00
Saved checkpoint for step 70890: ./temp/train/checkpoints/ckpt-7089
loss 0.03
Saved checkpoint for step 70900: ./temp/train/checkpoints/ckpt-7090
loss 0.01
Saved checkpoint for step 70910: ./temp/train/checkpoints/ckpt-7091
loss 0.03
Saved checkpoint for step 70920: ./temp/train/checkpoints/ckpt-7092
loss 0.03
Saved checkpoint for step 70930: ./temp/train/checkpoints/ckpt-7093
loss 0.01
Saved checkpoint for step 70940: ./temp/train/checkpoints/ckpt-7094
loss 0.00
Saved checkpoint for step 70950: ./temp/train/checkpoints/ckpt-7095
loss 0.00
Saved checkpoint for step 70960: ./temp/train/checkpoints/ckpt-7

Saved checkpoint for step 71900: ./temp/train/checkpoints/ckpt-7190
loss 0.01
Saved checkpoint for step 71910: ./temp/train/checkpoints/ckpt-7191
loss 0.06
Saved checkpoint for step 71920: ./temp/train/checkpoints/ckpt-7192
loss 0.01
Saved checkpoint for step 71930: ./temp/train/checkpoints/ckpt-7193
loss 0.00
Saved checkpoint for step 71940: ./temp/train/checkpoints/ckpt-7194
loss 0.06
Saved checkpoint for step 71950: ./temp/train/checkpoints/ckpt-7195
loss 0.02
Saved checkpoint for step 71960: ./temp/train/checkpoints/ckpt-7196
loss 0.00
Saved checkpoint for step 71970: ./temp/train/checkpoints/ckpt-7197
loss 0.01
Saved checkpoint for step 71980: ./temp/train/checkpoints/ckpt-7198
loss 0.20
Saved checkpoint for step 71990: ./temp/train/checkpoints/ckpt-7199
loss 0.00
Saved checkpoint for step 72000: ./temp/train/checkpoints/ckpt-7200
loss 0.06
Restored from ./temp/train/checkpoints/ckpt-7200
Saved checkpoint for step 72010: ./temp/train/checkpoints/ckpt-7201
loss 0.03
Saved checkpoin

Saved checkpoint for step 72950: ./temp/train/checkpoints/ckpt-7295
loss 0.00
Saved checkpoint for step 72960: ./temp/train/checkpoints/ckpt-7296
loss 0.00
Saved checkpoint for step 72970: ./temp/train/checkpoints/ckpt-7297
loss 0.03
Saved checkpoint for step 72980: ./temp/train/checkpoints/ckpt-7298
loss 0.06
Saved checkpoint for step 72990: ./temp/train/checkpoints/ckpt-7299
loss 0.00
Saved checkpoint for step 73000: ./temp/train/checkpoints/ckpt-7300
loss 0.10
Saved checkpoint for step 73010: ./temp/train/checkpoints/ckpt-7301
loss 0.01
Saved checkpoint for step 73020: ./temp/train/checkpoints/ckpt-7302
loss 0.09
Saved checkpoint for step 73030: ./temp/train/checkpoints/ckpt-7303
loss 0.04
Saved checkpoint for step 73040: ./temp/train/checkpoints/ckpt-7304
loss 0.01
Saved checkpoint for step 73050: ./temp/train/checkpoints/ckpt-7305
loss 0.19
Saved checkpoint for step 73060: ./temp/train/checkpoints/ckpt-7306
loss 0.00
Saved checkpoint for step 73070: ./temp/train/checkpoints/ckpt-7

Saved checkpoint for step 74000: ./temp/train/checkpoints/ckpt-7400
loss 0.02
Saved checkpoint for step 74010: ./temp/train/checkpoints/ckpt-7401
loss 0.02
Saved checkpoint for step 74020: ./temp/train/checkpoints/ckpt-7402
loss 0.13
Saved checkpoint for step 74030: ./temp/train/checkpoints/ckpt-7403
loss 0.00
Saved checkpoint for step 74040: ./temp/train/checkpoints/ckpt-7404
loss 0.04
Saved checkpoint for step 74050: ./temp/train/checkpoints/ckpt-7405
loss 0.03
Saved checkpoint for step 74060: ./temp/train/checkpoints/ckpt-7406
loss 0.22
Saved checkpoint for step 74070: ./temp/train/checkpoints/ckpt-7407
loss 0.32
Saved checkpoint for step 74080: ./temp/train/checkpoints/ckpt-7408
loss 0.02
Saved checkpoint for step 74090: ./temp/train/checkpoints/ckpt-7409
loss 0.00
Saved checkpoint for step 74100: ./temp/train/checkpoints/ckpt-7410
loss 0.05
Saved checkpoint for step 74110: ./temp/train/checkpoints/ckpt-7411
loss 0.07
Saved checkpoint for step 74120: ./temp/train/checkpoints/ckpt-7

In [235]:
# for epoch in range(EPOCHS):
#     ckpt.step.assign_add(1)

#     print(f'epoch: {epoch}')
#     losses.reset_states()
#     val_losses.reset_states()
#     for x_batch, y_batch in train_dataset:

#         loss = train_step(model, x_batch, y_batch, ce_loss, optimizer)
#         losses(loss)
# #         step += 1
    
#     save_path = manager.save()
#     print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))

# #     with file_writer.as_default():
# #         tf.summary.scalar('loss', losses.result(), step=epoch)
# #         tf.summary.image('Input images', x_batch, step=epoch)

#     print(losses.result())
        
#     for x_batch, y_batch in valid_dataset:
#         val_loss = valid_step(model, x_batch, y_batch, ce_loss)
#         val_losses(val_loss)
    
# #     with file_writer.as_default():
# #         tf.summary.scalar('val_loss', val_losses.result(), step=epoch)


In [236]:
tf.train.list_variables(manager.latest_checkpoint)

[('_CHECKPOINTABLE_OBJECT_GRAPH', []),
 ('net/bc1/.ATTRIBUTES/VARIABLE_VALUE', [10]),
 ('net/bc1/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE', [10]),
 ('net/bc1/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUTES/VARIABLE_VALUE', [10]),
 ('net/bc2/.ATTRIBUTES/VARIABLE_VALUE', [20]),
 ('net/bc2/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE', [20]),
 ('net/bc2/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUTES/VARIABLE_VALUE', [20]),
 ('net/bd3/.ATTRIBUTES/VARIABLE_VALUE', [128]),
 ('net/bd3/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE', [128]),
 ('net/bd3/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUTES/VARIABLE_VALUE', [128]),
 ('net/bd4/.ATTRIBUTES/VARIABLE_VALUE', [64]),
 ('net/bd4/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE', [64]),
 ('net/bd4/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUTES/VARIABLE_VALUE', [64]),
 ('net/bd5/.ATTRIBUTES/VARIABLE_VALUE', [10]),
 ('net/bd5/.OPTIMIZER_SLOT/optimizer/m/.ATTRIBUTES/VARIABLE_VALUE', [10]),
 ('net/bd5/.OPTIMIZER_SLOT/optimizer/v/.ATTRIBUT

In [237]:
def predict(inputs):
    predicted = model(inputs)
    return tf.nn.softmax(predicted)

In [238]:
for sample, label in test_dataset.batch(2).take(1):
    predictions = predict(sample)
    print(tf.argmax(predictions, axis=1), label)

tf.Tensor([9 2], shape=(2,), dtype=int64) tf.Tensor([9. 2.], shape=(2,), dtype=float32)


In [239]:
import os

In [240]:
model_dir = f'./temp/train/models/'
weights_path = os.path.join(model_dir, 'weights.h5')

In [241]:
pwd

'/home/prajwol/Documents/AI School Fuse/Deep Learning/ProjectAssignment'

In [242]:
weights_path

'./temp/train/models/weights.h5'

In [250]:
model.save_weights(weights_path)

In [245]:
model.compile()

In [246]:
model.fit(train_dataset)

    def call(self,x):
        
        # 1st Convolutional Layer
        #layer equals NHWC
        #First the Convolutional layer and the MaxPool
        x = tf.nn.conv2d(x, self.wc1, strides=[1,1,1,1], padding="VALID")
        x = tf.nn.bias_add(x,self.bc1)
        x = tf.nn.relu(x)
        x = tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="VALID")
        
                
        # 2nd Convolutional Layer
        x = tf.nn.conv2d(x, self.wc2, strides=[1, 1, 1, 1], padding="VALID")
        x = tf.nn.bias_add(x, self.bc2)
        x = tf.nn.relu(x)
        x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
        
        # Flattten out
        # N X Number of Nodes
        # Flatten()
        x = tf.reshape(x, (tf.shape(x)[0], -1))
        
        # Dense1
        x = tf.matmul(x, self.wd3)
        x = tf.nn.bias_add(x, self.bd3)
        x = tf.nn.relu(x)

        
        # Dense2
        x = tf.matmul(x, self.wd4)
        x = tf.nn.bi

    def call(self,x):
        
        # 1st Convolutional Layer
        #layer equals NHWC
        #First the Convolutional layer and the MaxPool
        x = tf.nn.conv2d(x, self.wc1, strides=[1,1,1,1], padding="VALID")
        x = tf.nn.bias_add(x,self.bc1)
        x = tf.nn.relu(x)
        x = tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="VALID")
        
                
        # 2nd Convolutional Layer
        x = tf.nn.conv2d(x, self.wc2, strides=[1, 1, 1, 1], padding="VALID")
        x = tf.nn.bias_add(x, self.bc2)
        x = tf.nn.relu(x)
        x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
        
        # Flattten out
        # N X Number of Nodes
        # Flatten()
        x = tf.reshape(x, (tf.shape(x)[0], -1))
        
        # Dense1
        x = tf.matmul(x, self.wd3)
        x = tf.nn.bias_add(x, self.bd3)
        x = tf.nn.relu(x)

        
        # Dense2
        x = tf.matmul(x, self.wd4)
        x = tf.nn.bi

    def call(self,x):
        
        # 1st Convolutional Layer
        #layer equals NHWC
        #First the Convolutional layer and the MaxPool
        x = tf.nn.conv2d(x, self.wc1, strides=[1,1,1,1], padding="VALID")
        x = tf.nn.bias_add(x,self.bc1)
        x = tf.nn.relu(x)
        x = tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="VALID")
        
                
        # 2nd Convolutional Layer
        x = tf.nn.conv2d(x, self.wc2, strides=[1, 1, 1, 1], padding="VALID")
        x = tf.nn.bias_add(x, self.bc2)
        x = tf.nn.relu(x)
        x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
        
        # Flattten out
        # N X Number of Nodes
        # Flatten()
        x = tf.reshape(x, (tf.shape(x)[0], -1))
        
        # Dense1
        x = tf.matmul(x, self.wd3)
        x = tf.nn.bias_add(x, self.bd3)
        x = tf.nn.relu(x)

        
        # Dense2
        x = tf.matmul(x, self.wd4)
        x = tf.nn.bi

ValueError: The model cannot be compiled because it has no loss to optimize.

In [247]:
model.summary()

Model: "le_net_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 74,954
Trainable params: 74,954
Non-trainable params: 0
_________________________________________________________________


In [248]:
model.predict("me.jpg")

IndexError: tuple index out of range